# Example Script for plotting Data from the HSPF Model.

This script has been tested with Python3.8

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import sys
import os
import probscale #Needed for frequency duration curve
from wdmtoolbox import wdmtoolbox as wdm
#from hspfbintoolbox import hspfbintoolbox as hbn #needed for reding data from the binary files.
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import subprocess
from scipy import stats
#from pandas.stats.api import ols
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import abline_plot

#Following code sets the common font size for all the plots
SMALL_SIZE = 9
MEDIUM_SIZE = 11
BIGGER_SIZE = 13

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

months = mdates.MonthLocator(interval=1)
yearticks = mdates.YearLocator(1)

#Common constants
ft2_to_acre = 43560
in_to_ft = 12
secondsInDay = 24*60*60

## Command line function call

This function is used to run the HSPF model though python.

In [ ]:
def run_command(command):
    '''Run the command and display the output from command line'''
    process = subprocess.Popen(command, stdout=subprocess.PIPE)
    while True:
        output = process.stdout.readline()
        output = output.decode('utf-8')
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    rc = process.poll()
    return rc


### This function is used to calculate exceedances to make FDC

In [ ]:
def exceed(df):
    '''Calculating exceedance values from a time series'''
    flow=df.values.tolist()
    #Making a list of the values
    flow=[x for x in flow if x==x] 
    #Removing all na values. 
    flow.sort(reverse=True)
    #sorting the values
    exceedance = (np.arange(1., len(flow) + 1) / len(flow))*100
    #calculating exceedances
    return flow, exceedance
    

## HSPF Model Run

In [ ]:
pHSPFexe=r'C:\BASINS45\models\HSPF\bin\WinHspfLt.exe' 
#Typically, HSPF software is located here. Change as needed
pHSPFModelPath = r'H:\HSPF-Training\HSPF200'
#This is where all the model files are. Change as needed
pUCI = os.path.join(pHSPFModelPath,'HSPF_Test240.uci')
#Name of the UCI file. Change as needed.
pEchoFile = 'HSPF_Test.ech'
#Name of the Echo file. Change as needed
process = subprocess.Popen([pHSPFexe,'-1', '-1', pUCI],
                           stdout=subprocess.PIPE,
                           universal_newlines=True)
#Run the HSPF model 
print('Running HSPF model. Please wait...')
process.wait()

try:
#Read echo file to check the model run
    echoFileContent=open(os.path.join(pHSPFModelPath,pEchoFile),'r')
    echoFileLines=echoFileContent.readlines()
    echoFileContent.close()
    if 'End of Job' in echoFileLines[len(echoFileLines)-1]:
        print('HSPF model ran successfully!')
    else:
        print('HSPF model did not run successfully')
        sys.exit()    
except:
    print('Could not open echo file. Check if any of the files are locked.')
    sys.exit()

## Creating Hydrology Graphs

### Creating hydrograph

In [ ]:
wdmfile = os.path.join(pHSPFModelPath,'HSPF_Test.wdm')
#Name of the WDM file that has input and output data
fig,ax = plt.subplots(2, figsize=(10,6), sharex=True,gridspec_kw={'height_ratios': [1, 2]})
#Defining the plot size. 10,6 is good for powerpoint, 6, 4.25 is good for the word document
dfsim = wdm.extract(wdmfile,9001)
#Extracting simulated flow data
dfsupy = wdm.extract(wdmfile,1)
#Extracting the precipitation data
ax[0].plot(dfsupy, color='red', linewidth=0.6)
#plotting precipitation on the top axis
ax[0].set_ylabel('Precipitation (in)')
ax[0].grid(which='both')
ax[0].set_title('HSPF Test')

ax[1].plot(dfsim, color='red', linewidth=0.6)
#Plotting simulated flow data on the bottom axis
ax[1].set_xlim((pd.Timestamp('2018-01-01'),pd.Timestamp('2018-01-31')))
#Setting the x-axis scale

ax[1].grid(which='both')
ax[1].set_ylabel('Flow (cfs)')
ax[1].legend(['Simulated'])
ax[1].set_xlabel('Year')

### Creating frequency duration curve

In [ ]:
fig, ax = plt.subplots(1,figsize=(10,6))
#Declaring plot size
dfsim = dfsim.resample('D').mean()
#Resampling the data to create a daily timeseries
flow, exceedance = exceed(dfsim)
#Using exceed function to calulate flow and exceedance
ax.plot(exceedance, flow, color='red', linewidth=0.6)
#plotting the graph
ax.set_yscale('log')
ax.set_xscale('prob')
ax.set_xlim([0.1, 99.9])
ax.set_ylim([0.1,10])
ax.set_ylabel('Average Daily Flow (cfs)')
ax.set_xlabel('Exceedance (%)')
ax.legend(['Simulated'])
ax.set_title('HSPF_Test')
ax.grid(which='both')